In [ ]:
%load_ext autoreload
import numpy as np

%autoreload 2
from zcu_tools.analysis.fluxdep import load_result, mA2flx, flx2mA
from zcu_tools.datasaver import load_data
from zcu_tools.analysis.dispersive.interactive_fit import (
    calculate_dispersive,
    plot_dispersive_with_onetone,
    search_proper_g,
)

In [ ]:
qub_name = "S001"

In [ ]:
loadpath = f"../../result/{qub_name}/params.json"
_, params, mA_c, period, allows = load_result(loadpath)

if "r_f" in allows:
    r_f = allows["r_f"]
    print(f"r_f = {r_f}")

# Plot with Onetone

In [ ]:
onetone_path = "../../Database/S001/2025/03/Data_0311/S001_OneTone_flux_003.hdf5"

signals, sp_fpts, sp_mAs = load_data(
    onetone_path, server_ip="005-writeboard", port=4999
)
sp_fpts = sp_fpts * 1e-9  # to GHz
sp_mAs = sp_mAs * 1e3  # to mA
signals = signals.T  # (sp_mAs, sp_fpts)

In [ ]:
r_f = 7.485
sp_flxs = mA2flx(sp_mAs, mA_c, period)

In [ ]:
flxs = np.linspace(sp_flxs.min(), sp_flxs.max(), 501)
mAs = flx2mA(flxs, mA_c, period)

In [ ]:
%matplotlib widget
finish_fn = search_proper_g(params, r_f, sp_flxs, sp_fpts, signals, g_bound=(0, 0.2))

In [ ]:
best_g = finish_fn()
best_g

In [ ]:
rf_0, rf_1 = calculate_dispersive(params, r_f, best_g, flxs)

In [ ]:
fig = plot_dispersive_with_onetone(
    r_f, best_g, mAs, flxs, rf_0, rf_1, sp_mAs, sp_flxs, sp_fpts, signals
)
fig.show()

In [ ]:
fig.write_html(f"../../result/{qub_name}/web/dispersive.html", include_plotlyjs="cdn")
fig.write_image(
    f"../../result/{qub_name}/image/dispersive.png", format="png", width=800, height=400
)